<a href="https://colab.research.google.com/github/Alexandr20i/Th_coding/blob/main/Th_coding_LR2_Belov_Vinogradov_Grigorev_Semenickhin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Лабораторная работа №2

In [23]:
import  numpy as np

# Часть 1

## 2.1 Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [24]:
# Заданные параметры
n, k, d = 7, 4, 3

# Создаем единичную матрицу I_k размером 4x4
identity_matrix = np.eye(k, dtype=int)

# Инициализация списка для хранения строк
generated_rows = []

# Генерация всех возможных строк длины d с двумя или более единицами
for i in range(2 ** d):
    # Преобразуем число в двоичное представление и создаем строку
    binary_row = [(i >> j) & 1 for j in range(d)]
    # Проверяем, что в строке как минимум две единицы
    if sum(binary_row) >= 2:
        generated_rows.append(binary_row)

# Убираем дубликаты
generated_rows = np.unique(generated_rows, axis=0)

# Выбираем первые 4 уникальные строки
selected_rows = generated_rows[:4]

# Формируем порождающую матрицу G
generator_matrix = np.hstack((identity_matrix, selected_rows))

print(f'Порождающая матрица линейного кода (7, 4, 3):\n{generator_matrix}')


Порождающая матрица линейного кода (7, 4, 3):
[[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]


## 2.2 Сформировать проверочную матрицу на основе порождающей

In [25]:
n_k = n - k  # Количество проверочных бит (3)

# Создание единичной матрицы I_{n-k} размером 3x3
I_n_k = np.eye(n_k, dtype=int)

# Формирование проверочной матрицы H
H = np.vstack((generated_rows, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## 2.3 Сформировать таблицу синдромов для всех однократных ошибок

In [34]:
def compute_syndrome(error_vector, parity_check_matrix):
    # Вычисляет синдром для данного вектора ошибки и проверочной матрицы
    syndrome = (error_vector @ parity_check_matrix) % 2
    return tuple(syndrome)

def generate_syndrome_table(parity_check_matrix):
    # Создает таблицу синдромов для заданной проверочной матрицы
    num_rows = parity_check_matrix.shape[0]
    syndrome_map = {}

    # Проходим по всем возможным позициям ошибок
    for i in range(num_rows):
        error_vector = np.zeros(num_rows, dtype=int)  # Создаем вектор ошибки с нулями
        error_vector[i] = 1  # Вносим ошибку в i-ю позицию
        syndrome = compute_syndrome(error_vector, parity_check_matrix)  # Вычисляем синдром
        syndrome_map[syndrome] = tuple(error_vector)  # Сохраняем синдром и соответствующий вектор ошибки

    return syndrome_map

# Пример использования
syndrome_table = generate_syndrome_table(H)
print("Таблица синдромов для однократных ошибок:")
print("Синдром\t\tВектор ошибки")
for syndrome in syndrome_table:
    print(f"{syndrome}\t{syndrome_table[syndrome]}")


Таблица синдромов для однократных ошибок:
Синдром		Вектор ошибки
(1, 1, 1, 1, 0)	(1, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 0, 0, 1)	(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 1, 0, 1)	(0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 1, 1, 0, 1)	(0, 0, 0, 1, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0)	(0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 1, 0, 0, 0)	(0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 1, 0, 0)	(0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 1, 0)	(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 1)	(0, 0, 0, 0, 0, 0, 0, 0, 1)


## 2.4 Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова


In [35]:
def fix_errors(parity_check_matrix, received_word, syndrome_lookup):
    # Исправляет ошибки в принятом кодовом слове с помощью таблицы синдромов.
    # Вычисляем синдром для принятого слова
    current_syndrome = tuple(np.dot(parity_check_matrix.T, received_word) % 2)

    # Проверяем, есть ли синдром в таблице
    if current_syndrome in syndrome_lookup:
        error_pattern = np.array(syndrome_lookup[current_syndrome], dtype=int)  # Получаем вектор ошибок
        corrected_word = (received_word + error_pattern) % 2  # Исправляем слово
        return corrected_word, error_pattern  # Возвращаем исправленное слово и вектор ошибок

    # Если синдром не найден, проверяем пары ошибок
    num_bits = len(received_word)
    for i in range(num_bits):
        for j in range(i + 1, num_bits):  # Избегаем повторяющихся пар
            # Создаем шаблон ошибки для текущей пары индексов
            error_pattern = np.zeros(num_bits, dtype=int)
            error_pattern[i] = 1  # Устанавливаем ошибку в i-ю позицию
            error_pattern[j] = 1  # Устанавливаем ошибку в j-ю позицию

            # Вычисляем новое слово и его синдром
            modified_word = (received_word + error_pattern) % 2
            new_syndrome = tuple(np.dot(parity_check_matrix.T, modified_word) % 2)

            # Проверяем новый синдром
            if new_syndrome in syndrome_lookup:
                corrected_word = (received_word + error_pattern) % 2  # Исправляем слово
                return corrected_word, error_pattern  # Возвращаем исправленное слово и вектор ошибок

    # Если ошибки не обнаружены, возвращаем исходное слово
    return received_word, None  # Возвращаем исходное слово и None для вектора ошибок

In [36]:
original_msg = np.random.randint(0, 2, size=k)
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
# внесем однократную ошибку
pos = np.random.randint(len(encoded_word))
corrupted_word = np.array([bit ^ (i == pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [0 1 0 0]
Кодированное сообщение: [0 1 0 0 1 1 0 0 1]
Сообщение с ошибкой в позиции 5: [0 1 0 0 1 0 0 0 1]
Обнаруженная ошибка: [0 0 0 0 0 1 0 0 0]
Исправленное сообщение: [0 1 0 0 1 1 0 0 1]
Проверка на равенство отправленного слова и полученного: True


## 2.5. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [37]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем двукратную ошибку
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [0 1 0 0]
Кодированное сообщение: [0 1 0 0 1 1 0 0 1]
Сообщение с ошибкой в позиции [7 1]: [0 0 0 0 1 1 0 1 1]
Обнаруженная ошибка: [1 0 1 0 0 0 0 0 0]
Исправленное сообщение: [1 0 1 0 1 1 0 1 1]
Проверка на равенство отправленного слова и полученного: False


# Часть 2

## 2.6. Сформировать порождающую матрицу линейного кода (n, k, 5).

In [38]:
# Параметры кода
n = 9  # длина кодового слова
k = 4  # длина информационного слова
n_k = n - k
# Примерная матрица P
X = np.array([[1, 1, 1, 1, 0],
              [1, 1, 0, 0, 1],
              [1, 0, 1, 0, 1],
              [0, 1, 1, 0, 1]])
I_k = np.eye(k, dtype=int)
G = np.hstack((I_k, X))
print("Порождающая матрица G:\n", G)

Порождающая матрица G:
 [[1 0 0 0 1 1 1 1 0]
 [0 1 0 0 1 1 0 0 1]
 [0 0 1 0 1 0 1 0 1]
 [0 0 0 1 0 1 1 0 1]]


## 2.7 Сформировать проверочную матрицу на основе порождающей.

In [39]:
n_k = n - k  # Количество проверочных бит (3)

# Создание единичной матрицы I_{n-k} размером 3x3
I_n_k = np.eye(n_k, dtype=int)

# Формирование проверочной матрицы H
H = np.vstack((X, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[1 1 1 1 0]
 [1 1 0 0 1]
 [1 0 1 0 1]
 [0 1 1 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


## 2.8 Сформировать таблицу синдромов для всех однократных и двукратных ошибок.

In [40]:
def build_syndrome_table(H):
    n = H.shape[1]  # Количество столбцов (размер кодового слова)

    syndromes = {}

    # Однократные ошибки
    for i in range(n):
        error_vec = np.zeros(n, dtype=int)
        error_vec[i] = 1
        S = calculate_syndrome(error_vec, H.T)
        syndromes[S] = tuple(error_vec)

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            error_vec = np.zeros(n, dtype=int)
            error_vec[i] = 1
            error_vec[j] = 1
            S = calculate_syndrome(error_vec, H.T)
            syndromes[S] = tuple(error_vec)

    return syndromes

print("Таблица синдромов для однократных и двухкратных ошибок:")
syndrome_table = build_syndrome_table(H.T)
print("Синдром\t\t\tВектор ошибки")
for syndrome in syndrome_table:
    print(f"{syndrome}\t\t{syndrome_table[syndrome]}")


Таблица синдромов для однократных и двухкратных ошибок:
Синдром			Вектор ошибки
(1, 1, 1, 1, 0)		(1, 0, 0, 0, 0, 0, 0, 0, 0)
(1, 1, 0, 0, 1)		(0, 1, 0, 0, 0, 0, 0, 0, 0)
(1, 0, 1, 0, 1)		(0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 1, 1, 0, 1)		(0, 0, 0, 1, 0, 0, 0, 0, 0)
(1, 0, 0, 0, 0)		(0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 1, 0, 0, 0)		(0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 1, 0, 0)		(0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 1, 0)		(0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 1)		(0, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 1, 1, 1)		(1, 1, 0, 0, 0, 0, 0, 0, 0)
(0, 1, 0, 1, 1)		(1, 0, 1, 0, 0, 0, 0, 0, 0)
(1, 0, 0, 1, 1)		(1, 0, 0, 1, 0, 0, 0, 0, 0)
(0, 1, 1, 1, 0)		(1, 0, 0, 0, 1, 0, 0, 0, 0)
(1, 0, 1, 1, 0)		(1, 0, 0, 0, 0, 1, 0, 0, 0)
(1, 1, 0, 1, 0)		(1, 0, 0, 0, 0, 0, 1, 0, 0)
(1, 1, 1, 0, 0)		(1, 0, 0, 0, 0, 0, 0, 1, 0)
(1, 1, 1, 1, 1)		(1, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 1, 1, 0, 0)		(0, 0, 0, 0, 0, 1, 1, 0, 0)
(1, 0, 1, 0, 0)		(0, 0, 0, 0, 1, 0, 1, 0, 0)
(0, 1, 0, 0, 1)		(0, 0, 0, 0, 0, 1, 0, 0, 1)
(1, 0, 0, 0, 1)		(0,

## 2.9. Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [41]:
original_msg = np.random.randint(0, 2, size=k)
print(f"Оригинальное сообщение: {original_msg}")

encoded_word =  np.dot(original_msg, G) % 2
print(f"Кодированное сообщение: {encoded_word}")
# внесем однократную ошибку
pos = np.random.randint(len(encoded_word))
corrupted_word = np.array([bit ^ (i == pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [0 1 0 1]
Кодированное сообщение: [0 1 0 1 1 0 1 0 0]
Сообщение с ошибкой в позиции 0: [1 1 0 1 1 0 1 0 0]
Обнаруженная ошибка: [1 0 0 0 0 0 0 0 0]
Исправленное сообщение: [0 1 0 1 1 0 1 0 0]
Проверка на равенство отправленного слова и полученного: True


## 2.10. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [42]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем двукратную ошибку
pos = np.random.choice(len(encoded_word), size=2, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [0 1 0 1]
Кодированное сообщение: [0 1 0 1 1 0 1 0 0]
Сообщение с ошибкой в позиции [8 1]: [0 0 0 1 1 0 1 0 1]
Обнаруженная ошибка: [0 0 0 0 1 1 0 0 0]
Исправленное сообщение: [0 0 0 1 0 1 1 0 1]
Проверка на равенство отправленного слова и полученного: False


## 2.11. Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [43]:
print(f"Оригинальное сообщение: {original_msg}")
print(f"Кодированное сообщение: {encoded_word}")
# внесем трехкратную ошибку
pos = np.random.choice(len(encoded_word), size=3, replace=False)
corrupted_word = np.array([bit ^ (i in pos) for i, bit in enumerate(encoded_word)])
print(f"Сообщение с ошибкой в позиции {pos}: {corrupted_word}")

corrected_word, detected_error = fix_errors(H, corrupted_word, syndrome_table)

if detected_error is not None:
    print(f"Обнаруженная ошибка: {detected_error}")
else:
    print("Ошибки не обнаружено.")

print(f"Исправленное сообщение: {corrected_word}")

print(f"Проверка на равенство отправленного слова и полученного: {np.array_equal(original_msg, corrected_word[:k])}")

Оригинальное сообщение: [0 1 0 1]
Кодированное сообщение: [0 1 0 1 1 0 1 0 0]
Сообщение с ошибкой в позиции [4 3 5]: [0 1 0 0 0 1 1 0 0]
Обнаруженная ошибка: [0 0 1 0 0 0 0 0 0]
Исправленное сообщение: [0 1 1 0 0 1 1 0 0]
Проверка на равенство отправленного слова и полученного: False
